In [1]:
!bq --location=US mk --dataset college_modeled

Dataset 'cs327e-sp2020:college_modeled' successfully created.


In [3]:
%%bigquery
create table college_modeled.Student as
select distinct sid, fname, lname, dob, 'CUR' as status
from college_staging.Current_Students
union all
select distinct sid, fname, lname, cast(dob as string) as dob, 'PRO' as status
from college_staging.New_Students;

""


In [4]:
%%bigquery
select * from college_modeled.Student order by sid limit 12;

,sid,fname,lname,dob,status
0,aprilz,April,Lopez,01/10/2000,CUR
1,aprilz,April,Lopez,2000-01-10,PRO
2,bzen26,Biswa,Zen,04/22/1998,CUR
3,dchen,David,Chen,2000-08-22,PRO
4,jc,James,Cowe,2000-04-22,CUR
5,jerryh,Jerry,Hargrove,1999-01-03,CUR
6,jerryh,Jerry,Hargrove,01/03/1999,CUR
7,kev18,Kevin,Lin,05/10/1999,CUR
8,kgomez9,Katie,Gomez,2001-03-03,PRO
9,natea,Nathan,Anderson,2000-04-08,PRO


In [5]:
%%bigquery
create table college_modeled.Teacher as
select distinct tid, instructor, dept
from college_staging.Classes
where tid is not null;

""


In [6]:
%%bigquery
select * from college_modeled.Teacher order by tid limit 12;

,tid,instructor,dept
0,abraham,Prof. Sarah Abraham,CS
1,bulko,bill bulko,Computer Science
2,cannata,"Cannata, Philip",CS
3,cannata,PHILIP CANNATA,CS
4,fares,Fares Fraij,CS
5,koch,"Koch, Hans",Math
6,mitra,"Mitra, Shyamal",CS
7,mitra,"MITRA, SHYAMAL",Computer Science
8,mueller,"MUELLER, PETER",Math
9,neeman,JOE NEEMAN,Mathematics


In [7]:
%%bigquery
create table college_modeled.Class as
select distinct cno, cname, credits
from college_staging.Classes
where cno is not null;

""


In [8]:
%%bigquery
select * from college_modeled.Class order by cno limit 12;

,cno,cname,credits
0,CS313E,Elements of Software Design,3
1,CS326E,Elements of Networking,3
2,CS327E,Elements of Databases,3
3,CS329E,Elements of Mobile Computing,3
4,CS329E,Elements of Web Programming,3
5,CS331E,Elements of Software Engineering,3
6,CS347,Data Management,3
7,CS373,CS373: Software Engineering,3
8,M328K,Intro to Number Theory,3
9,M358K,Applied Statistics,3


In [9]:
%%bigquery
select count(*) from college_modeled.Class

,f0_
0,11


In [10]:
%%bigquery
select count(distinct cno) from college_modeled.Class

,f0_
0,10


In [11]:
%%bigquery
create table college_modeled.Teaches as
select distinct tid, cno
from college_staging.Classes
where tid is not null
and cno is not null;

""


In [12]:
%%bigquery
select * from college_modeled.Teaches order by tid limit 12;

,tid,cno
0,abraham,CS329E
1,bulko,CS313E
2,cannata,CS347
3,cannata,CS326E
4,fares,CS331E
5,koch,M328K
6,mitra,CS313E
7,mitra,CS329E
8,mueller,M362K
9,neeman,M362K


In [13]:
%%bigquery
create table college_modeled.Takes as
select distinct sid, cno, grade
from college_staging.Current_Students
where sid is not null
and cno is not null;

""


In [14]:
%%bigquery
select * from college_modeled.Takes order by sid limit 12;

,sid,cno,grade
0,bzen26,CS313E - Fall18,B+
1,jc,CS327E,None
2,jc,CS331E,None
3,jc,CS313E,None
4,jerryh,CS327E,B
5,jerryh,CS329E,A-
6,jerryh,CS327E,None
7,kev18,CS329E,None
8,paulg,CS313E,None
9,paulg,CS326E - Fall18,A-


### Run beam pipelines

In [1]:
%run Student_beam1.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/io/gcp/bigquery.py:1421: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []
INFO:apache_beam.runners.direct.direct_runner:Running pipeline with DirectRunner.
INFO:apache_beam.internal.gcp.auth:Setting socket default timeout to 60 seconds.
INFO:apache_beam.internal.gcp.auth:socket default timeout is 60.0 seconds.
INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:apache_beam.io.gcp.bigquery_tools:Using location 'US' from table <TableReference
 datasetId: 'college_modeled'
 projectId: 'cs327e-sp2020'
 tableId: 'Student'> referenced by query SELECT sid, fname, lname, dob FROM college_modeled.Student limit 50
INFO:apache_beam.io.filebasedsink:Starting finalize_write threads with num_shards: 1 (skipped: 0), batches: 1, num_threads: 1


current dob: 2000-04-22
current dob: 2000-08-22
current dob: 05/10/1999
new dob: 1999-05-10
current dob: 2000-04-08
current dob: 09/17/2000
new dob: 2000-09-17
current dob: 2000-09-17
current dob: 2000-01-10
current dob: 01/10/2000
new dob: 2000-01-10
current dob: 04/22/1998
new dob: 1998-04-22
current dob: 1999-01-03
current dob: 01/03/1999
new dob: 1999-01-03
current dob: 1999-07-31
current dob: 2000-11-07
current dob: 2001-03-03
current dob: 2000-10-01
current dob: 2000-10-01
student_record: {'dob': '2000-04-22', 'sid': 'jc', 'lname': 'Cowe', 'fname': 'James'}
student_record: {'dob': '2000-11-07', 'sid': 'ryan22', 'lname': 'Watson', 'fname': 'Ryan'}
student_record: {'dob': '2000-04-08', 'sid': 'natea', 'lname': 'Anderson', 'fname': 'Nathan'}
student_record: {'dob': '1998-04-22', 'sid': 'bzen26', 'lname': 'Zen', 'fname': 'Biswa'}
student_record: {'dob': '2000-10-01', 'sid': 'sudeepa4', 'lname': 'Roy', 'fname': 'Sudeepa'}
student_record: {'dob': '2001-03-03', 'sid': 'kgomez9', 'lname'

INFO:apache_beam.io.filebasedsink:Renamed 1 shards in 0.10 seconds.
INFO:apache_beam.io.filebasedsink:Starting finalize_write threads with num_shards: 1 (skipped: 0), batches: 1, num_threads: 1
INFO:apache_beam.io.filebasedsink:Renamed 1 shards in 0.10 seconds.
INFO:apache_beam.io.filebasedsink:Starting finalize_write threads with num_shards: 1 (skipped: 0), batches: 1, num_threads: 1
INFO:apache_beam.io.filebasedsink:Renamed 1 shards in 0.10 seconds.
INFO:apache_beam.io.gcp.bigquery_tools:Created table cs327e-sp2020.college_modeled.Student_Beam with schema <TableSchema
 fields: [<TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'sid'
 type: 'STRING'>, <TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'fname'
 type: 'STRING'>, <TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'lname'
 type: 'STRING'>, <TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'dob'
 type: 'DATE'>]>. Result: <Table
 creationTime: 1582595717967
 etag: 'VrTVRIEV7ljcwqldBsIkIQ=='
 id: 'cs327e-sp

In [2]:
%%bigquery
drop table college_modeled.Student_Beam

""


In [ ]:
%run Student_beam2.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/io/gcp/bigquery.py:1421: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []
INFO:apache_beam.runners.direct.direct_runner:Running pipeline with DirectRunner.
INFO:apache_beam.internal.gcp.auth:Setting socket default timeout to 60 seconds.
INFO:apache_beam.internal.gcp.auth:socket default timeout is 60.0 seconds.
INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:apache_beam.io.gcp.bigquery_tools:Using location 'US' from table <TableReference
 datasetId: 'college_modeled'
 projectId: 'cs327e-sp2020'
 tableId: 'Student'> referenced by query SELECT sid, fname, lname, dob, status FROM college_modeled.Student limit 50
INFO:apache_beam.io.filebasedsink:Starting finalize_write threads with num_shards: 1 (skipped: 0), batches: 1, num_threads: 1


current dob: 2000-04-22
current dob: 2000-08-22
current dob: 05/10/1999
new dob: 1999-05-10
current dob: 2000-04-08
current dob: 2000-09-17
current dob: 09/17/2000
new dob: 2000-09-17
current dob: 2000-01-10
current dob: 01/10/2000
new dob: 2000-01-10
current dob: 04/22/1998
new dob: 1998-04-22
current dob: 1999-01-03
current dob: 01/03/1999
new dob: 1999-01-03
current dob: 1999-07-31
current dob: 2000-11-07
current dob: 2001-03-03
current dob: 2000-10-01
current dob: 2000-10-01
student_record: {'sid': 'kev18', 'fname': 'Kevin', 'status': 'CUR', 'dob': '1999-05-10', 'lname': 'Lin'}
student_record: {'sid': 'kgomez9', 'fname': 'Katie', 'status': 'PRO', 'dob': '2001-03-03', 'lname': 'Gomez'}
student_record: {'sid': 'sudeepa4', 'fname': 'Sudeepa', 'status': 'CUR', 'dob': '2000-10-01', 'lname': 'Roy'}
student_record: {'sid': 'aprilz', 'fname': 'April', 'status': 'CUR', 'dob': '2000-01-10', 'lname': 'Lopez'}
student_record: {'sid': 'nitinm', 'fname': 'Nitin', 'status': 'PRO', 'dob': '1999-07

INFO:apache_beam.io.filebasedsink:Renamed 1 shards in 0.11 seconds.
INFO:apache_beam.io.filebasedsink:Starting finalize_write threads with num_shards: 1 (skipped: 0), batches: 1, num_threads: 1
INFO:apache_beam.io.filebasedsink:Renamed 1 shards in 0.10 seconds.
INFO:apache_beam.io.gcp.bigquery_tools:Created table cs327e-sp2020.college_modeled.Student_Beam with schema <TableSchema
 fields: [<TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'sid'
 type: 'STRING'>, <TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'fname'
 type: 'STRING'>, <TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'lname'
 type: 'STRING'>, <TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'dob'
 type: 'DATE'>, <TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'status'
 type: 'STRING'>]>. Result: <Table
 creationTime: 1582653961141
 etag: 'EEoEQS4rjn7WOF2dJ08prw=='
 id: 'cs327e-sp2020:college_modeled.Student_Beam'
 kind: 'bigquery#table'
 lastModifiedTime: 1582653961183
 location: 'US'
 num